## Homework 1

## Installing dependencies

> **The submission (zip file) consists of the notebook.ipynb + trace1 ... traceN.nc files** (file, export project as ZIP)

In [ ]:
%pip install -U pymc numpy scipy

## Estimating Purchases

In [ ]:
import pymc as pm
import pandas as pd
import io
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import arviz as az
%matplotlib inline

You have raw purchase data (**obs**) from three grocery stores (**store**) for the same product (zero means a customer did not buy the product). Groceries are of different size so number of observations are different. You need to compare which grocery is more efficient in sales.


1. (5 pt) Propose and motivate a likelihood for the problem
    * likelihood with negative support = -5 points
2. (10 pt) Propose and motivate a probabilistic model (non-hierarchical) for the problem. Some parameters may be shared, some independent if you want.
    0. (2.5 pt) Write LaTeX equation for the model (displaying model is ok)
    1. (2.5 pt) Motivate priors, under the only assumption that purchases are at orders of 100s and no more than 500
    2. (2.5 pt) Make a prior predictive check to confirm your choice
    3. (2.5 pt) Sample from the model, plot traces compare it with prior, comment on it (e.g. infulence of data size).
    4. **total score for the section is multiplied by (1 - divergent samples / total samples)**
    5. Save trace into traceN.nc
3. (10 pt) Propose a hierarchical model for the problem. Put hierarchy on only one model parameter (you choose which one, prefer mean)
    
    same as 2.
    
4. (10 pt) Propose a hierarchical model for the problem. Put hierarchy on 2 model parameters (you choose which additional one, prefer p)
    
    same as 2

5. (5 pt) Perform a model comparison study (look into [`az.compare`](https://arviz-devs.github.io/arviz/api/generated/arviz.compare.html)), you can use the models you have (at least 2)
    1. Interpret the result and choose the model, elaborate on warnings
6. (5 pt) Business Metric (bonus)
    1. Propose and motivate a comparison metric between groceries to say (A is better than B)
    2. Evaluate the metric on the the best model you have
7. (5 pt) Make a prediction for a new store, you need model from (4) (bonus)

(max 40 pt)

**Important**
> Use quoting for the main points of your solution

In [ ]:
raw_data = io.StringIO('obs,store\n117,0\n0,0\n0,0\n134,0\n0,0\n0,0\n0,0\n0,0\n122,0\n0,0\n108,0\n0,0\n0,0\n0,0\n0,0\n124,0\n0,0\n0,0\n113,0\n118,0\n0,0\n105,0\n114,0\n0,0\n118,0\n106,0\n0,0\n100,0\n0,0\n0,0\n123,0\n0,0\n0,0\n0,0\n0,0\n129,0\n0,0\n117,0\n0,0\n0,0\n0,0\n0,0\n0,0\n115,0\n123,0\n0,0\n0,0\n0,0\n137,0\n0,0\n0,0\n0,0\n0,0\n0,0\n136,0\n0,0\n107,0\n0,0\n0,0\n0,0\n0,0\n115,0\n118,0\n0,0\n105,0\n105,0\n97,0\n121,0\n0,0\n0,0\n0,0\n124,0\n114,0\n0,0\n101,0\n0,0\n0,0\n0,0\n121,0\n0,0\n0,0\n0,0\n119,0\n0,0\n108,0\n121,0\n0,0\n0,0\n116,0\n0,0\n0,0\n0,0\n0,0\n0,0\n0,0\n0,0\n127,0\n0,0\n0,0\n106,0\n0,0\n0,0\n0,0\n131,0\n0,0\n0,0\n128,0\n122,0\n0,0\n0,0\n0,0\n0,0\n0,0\n0,0\n102,0\n119,0\n0,0\n0,0\n0,0\n123,0\n0,0\n0,0\n0,0\n120,0\n0,1\n0,1\n0,1\n0,1\n0,1\n121,1\n0,1\n0,1\n0,1\n123,1\n0,1\n106,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n142,1\n0,1\n0,1\n0,1\n131,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,1\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n81,2\n0,2\n0,2\n73,2\n0,2\n0,2\n0,2\n69,2\n0,2\n94,2\n0,2\n91,2\n0,2\n0,2\n98,2\n0,2\n79,2\n90,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n102,2\n0,2\n87,2\n0,2\n81,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n86,2\n97,2\n0,2\n0,2\n0,2\n76,2\n0,2\n83,2\n0,2\n0,2\n88,2\n0,2\n86,2\n0,2\n0,2\n99,2\n0,2\n0,2\n86,2\n86,2\n0,2\n93,2\n0,2\n0,2\n0,2\n90,2\n0,2\n81,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n91,2\n0,2\n0,2\n67,2\n107,2\n0,2\n0,2\n0,2\n0,2\n101,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n75,2\n112,2\n0,2\n0,2\n0,2\n0,2\n0,2\n91,2\n0,2\n0,2\n0,2\n81,2\n108,2\n92,2\n0,2\n0,2\n0,2\n90,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n93,2\n0,2\n0,2\n0,2\n96,2\n0,2\n0,2\n0,2\n0,2\n94,2\n0,2\n90,2\n0,2\n0,2\n0,2\n0,2\n94,2\n0,2\n0,2\n0,2\n0,2\n72,2\n0,2\n97,2\n0,2\n82,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n105,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n99,2\n90,2\n0,2\n76,2\n92,2\n0,2\n0,2\n0,2\n0,2\n0,2\n100,2\n113,2\n0,2\n0,2\n0,2\n100,2\n0,2\n91,2\n0,2\n88,2\n0,2\n0,2\n0,2\n83,2\n0,2\n0,2\n82,2\n0,2\n84,2\n101,2\n0,2\n0,2\n0,2\n0,2\n102,2\n0,2\n0,2\n0,2\n99,2\n91,2\n0,2\n95,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n73,2\n0,2\n80,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n90,2\n0,2\n0,2\n0,2\n0,2\n95,2\n0,2\n91,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n92,2\n0,2\n0,2\n97,2\n70,2\n102,2\n0,2\n0,2\n0,2\n87,2\n111,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n73,2\n79,2\n99,2\n0,2\n0,2\n88,2\n88,2\n0,2\n0,2\n94,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n94,2\n0,2\n88,2\n0,2\n0,2\n0,2\n0,2\n0,2\n80,2\n90,2\n0,2\n0,2\n0,2\n87,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n105,2\n0,2\n0,2\n88,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n71,2\n0,2\n103,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n96,2\n0,2\n0,2\n0,2\n94,2\n0,2\n79,2\n110,2\n0,2\n0,2\n0,2\n90,2\n0,2\n0,2\n0,2\n101,2\n0,2\n95,2\n94,2\n0,2\n0,2\n0,2\n0,2\n0,2\n85,2\n0,2\n100,2\n92,2\n0,2\n0,2\n0,2\n110,2\n0,2\n0,2\n0,2\n0,2\n78,2\n0,2\n74,2\n0,2\n103,2\n0,2\n0,2\n0,2\n0,2\n90,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n90,2\n105,2\n0,2\n0,2\n0,2\n103,2\n0,2\n0,2\n0,2\n0,2\n98,2\n89,2\n105,2\n0,2\n0,2\n93,2\n0,2\n93,2\n0,2\n0,2\n0,2\n0,2\n88,2\n88,2\n0,2\n0,2\n91,2\n0,2\n89,2\n0,2\n0,2\n100,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n75,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n85,2\n0,2\n83,2\n0,2\n0,2\n80,2\n0,2\n0,2\n0,2\n0,2\n89,2\n0,2\n0,2\n76,2\n0,2\n89,2\n0,2\n0,2\n0,2\n0,2\n0,2\n88,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n101,2\n0,2\n112,2\n92,2\n0,2\n0,2\n0,2\n0,2\n0,2\n93,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n0,2\n95,2\n0,2\n0,2\n94,2\n0,2\n0,2\n0,2\n0,2\n0,2\n104,2\n0,2\n89,2\n0,2\n88,2\n0,2\n0,2\n0,2\n0,2\n86,2\n82,2\n0,2\n0,2\n0,2\n0,2\n0,2\n91,2\n0,2\n83,2\n0,2\n81,2\n0,2\n0,2\n87,2\n0,2\n0,2\n0,2\n83,2\n0,2\n94,2\n0,2\n0,2\n81,2\n0,2\n0,2\n0,2\n87,2\n0,2\n0,2\n108,2\n')
data = pd.read_csv(raw_data)

In [ ]:
data

## 1. Data description
1. Propose and motivate a likelihood for the problem

## 2. Probabilistic model

2. Propose and motivate a probabilistic model (non-hierarchical) for the problem. Some parameters may be shared, some independent if you want.
    
    0. Write LaTeX equation for the model (displaying latex model is ok)
    1. Motivate priors, under the only assumption that purchases are at orders of 100s and no more than 500
    2. Make a prior predictive check to confirm your choice
    3. Sample from the model, plot traces compare it with prior, comment on it (e.g. infulence of data size).

**hints:**

1. explore `at.stack` to fix shape issues with mixture probability, (if you use mixtures)
2. (ADDED) transform=pm.distributions.transforms.log in case of mixtures in likelihood (warning can appear otherwise)

In [ ]:
with pm.Model(coords=dict(store=range(3))) as model:
    ...

In [ ]:
with model:
    prior = pm.sample_prior_predictive()

In [ ]:
with model:
    trace = pm.sample()
    trace.extend(prior)

## 3. Hierarchical Model (1 parameter)
3. Propose a hierarchical model for the problem. Put hierarchy on only one model parameter (you choose which one, prefer mean)

**hints:**

1. Use non centered parametrization
2. Use pm.Deterministic to track reparametrized parameters

In [ ]:
with pm.Model(coords=dict(store=range(3))) as model1:
    ...

In [ ]:
with model1:
    prior1 = pm.sample_prior_predictive()

In [ ]:
with model1:
    trace1 = pm.sample()
    trace1.extend(prior1)

## 4. Hierarchical Model (2 parameters)
4. Propose a hierarchical model for the problem. Put hierarchy on 2 model parameters (you choose which additional one, prefer p)

**hint:**

For Hierarchical probs:

Use LogitNormal distribution (preferred)

1. simple to interpret
2. easier to parametrize (same you do normal dist but with a transform)

In [ ]:
with pm.Model(coords=dict(store=range(3))) as model2:
    ...

In [ ]:
with model2:
    prior2 = pm.sample_prior_predictive()

In [ ]:
with model2:
    trace2 = pm.sample()
    trace2.extend(prior2)

## 5. Model Comparison study

5. Perform a model comparison study (look into az.compare)

    1. Interpret the result and choose the best model
    2. Elaborate on Warnings, if there are any

In [ ]:
res = az.compare(dict(simple=trace, hierarchy1=trace1, hierarchy2=trace2))

In [ ]:
res